## Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
%cd rapidsai-csp-utils
!git checkout 22.12-patch
%cd ..
!python rapidsai-csp-utils/colab/env-check.py
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 2.14 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/rapidsai-csp-utils
Branch '22.12-patch' set up to track remote branch '22.12-patch' from 'origin'.
Switched to a new branch '22.12-patch'
/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/env-check.py", line 26, in <module>
    gpu_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0)).decode('UTF-8')
AttributeError: 'str' object has no attribute 'decode'. Did you mean: 'encode'?
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 26, in <module>
    gpu_name =

In [3]:
%%bash
# Install the Merlin Framework
pip install -Uq git+https://github.com/NVIDIA-Merlin/models.git@release-23.04
pip install -Uq git+https://github.com/NVIDIA-Merlin/nvtabular.git@release-23.04
pip install -Uq git+https://github.com/NVIDIA-Merlin/core.git@release-23.04
# pip install -Uq git+https://github.com/NVIDIA-Merlin/system.git@release-23.04
# pip install -Uq git+https://github.com/NVIDIA-Merlin/dataloader.git@release-23.04
pip install -Uq git+https://github.com/NVIDIA-Merlin/Transformers4Rec.git@release-23.04
# pip install -Uq xgboost lightfm implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.5/985.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
merlin-dataloader 23.6.0 requires merlin-core<23.07,>=23.06, but you have merlin-core 23.4.0+4.gc5c9bc2 which is incompatible.


## Load Data
Reference
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb

In [4]:
import os
import shutil
import numpy as np
from nvtabular import ops
from merlin.schema.tags import Tags
import merlin.io
import merlin.models.tf as mm

import nvtabular as nvt

from os import path

# Get dataframe library - cudf or pandas
from merlin.core.dispatch import get_lib
pd = get_lib()

/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [5]:
DATA_DIR = os.environ.get(
    "DATA_DIR", os.path.expanduser("/content/drive/MyDrive/ml-twotower-model/data/")
)

In [6]:
import glob

USE_AUGMENTED_DATASET= True
DATA_VERSION = 'v3'

_TRAIN,_VALID,_TEST = ('train_aug', 'valid_aug', 'test_aug') if USE_AUGMENTED_DATASET else ('train', 'valid', 'test')

TRAIN_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _TRAIN + f'_{DATA_VERSION}', "*.parquet")))
VALID_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _VALID + f'_{DATA_VERSION}', "*.parquet")))
TEST_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _TEST + f'_{DATA_VERSION}', "*.parquet")))
TRAIN_PATHS, VALID_PATHS, TEST_PATHS

(['/content/drive/MyDrive/ml-twotower-model/data/train_aug_v3/part_0.parquet'],
 ['/content/drive/MyDrive/ml-twotower-model/data/valid_aug_v3/part_0.parquet'],
 ['/content/drive/MyDrive/ml-twotower-model/data/test_aug_v3/part_0.parquet'])

In [7]:
train = merlin.io.Dataset(
    TRAIN_PATHS, engine="parquet"
)
valid = merlin.io.Dataset(
    VALID_PATHS, engine="parquet"
)

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [8]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.freq_threshold,properties.max_size,properties.num_buckets,properties.start_index,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,1561,movie_id,NaN,NaN
1,user_id,"(Tags.CATEGORICAL, Tags.ID, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,74.0,944.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,943,user_id,NaN,NaN
2,gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,3.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,22.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,16.0,20.0,.//categories/unique.genres.parquet,0.0,0.0,None,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,.//categories/unique.synopsis.parquet,0.0,0.0,None,0.0,0,1561,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,.//categories/unique.title.parquet,0.0,0.0,None,0.0,0,1561,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,178.0,4504.0,.//categories/unique.cast.parquet,0.0,0.0,None,0.0,0,4503,cast,0.0,NaN


In [9]:
#sanity check
train.to_ddf().compute()

,movie_id,user_id,gender,occupation,genres,rating,movie_id_raw,synopsis,title,cast
0,396,106,2,2,"[12, 5]",0.0,217,402,404,"[564, 305, 199, 1193, 1259, 1286]"
1,16,7,2,1,"[1, 5]",0.0,237,16,16,"[16, 226, 200, 264]"
2,318,684,1,9,[16],0.0,661,318,319,"[966, 247, 315, 1101, 1333]"
3,403,12,1,5,"[14, 10]",0.0,969,399,399,"[1547, 1544, 1556, 696, 1557, 1542, 1539]"
4,400,175,1,9,"[2, 5]",1.0,1048,396,397,"[1042, 1456, 303, 463, 1558]"
...,...,...,...,...,...,...,...,...,...,...
70197,20,45,1,4,"[2, 7]",0.0,204,20,20,"[146, 61, 270, 280, 308]"
70198,87,36,1,3,"[11, 4]",1.0,322,87,87,"[432, 386, 174, 594, 435]"
70199,98,480,1,4,[1],1.0,508,98,98,"[224, 500, 138, 130, 650, 654]"
70200,97,79,2,6,[1],1.0,134,96,96,"[426, 366, 632, 644, 640, 636, 631, 641]"


In [10]:
# select only trainable features and exclude the target
train.schema = schema
valid.schema = schema

In [11]:
assert len(schema.select_by_tag(Tags.TARGET).column_names) == 0

## Augment with custom pre-trained embeddings

### Load embeddings

In [12]:
embeddings = pd.read_parquet(os.path.join(DATA_DIR, 'embeddings_index_v2.parquet')).set_index('movie_id')
embeddings.shape

(1561, 2)

In [13]:
embeddings

,title_embeddings,synopsis_embeddings
movie_id,,
243,"[-0.13162778317928314, -0.10936661064624786, -...","[-0.22289082407951355, 0.27888065576553345, 0...."
39,"[0.08139041066169739, -0.16367477178573608, -0...","[-0.11132552474737167, -0.04103025048971176, 0..."
55,"[0.15842819213867188, -0.032023221254348755, -...","[0.07356611639261246, -0.005173327401280403, 0..."
45,"[-0.13973252475261688, -0.19878588616847992, -...","[0.07982735335826874, -0.12655968964099884, 0...."
285,"[-0.18689653277397156, -0.19982194900512695, 0...","[0.06059129536151886, 0.12666839361190796, 0.2..."
...,...,...
1457,"[-0.06935157626867294, -0.3555041551589966, -0...","[-0.04539063572883606, 0.08155060559511185, 0...."
1532,"[-0.10045406222343445, -0.2019217312335968, -0...","[-0.1678621768951416, 0.07765717059373856, 0.0..."
1442,"[-0.18853621184825897, -0.4115959107875824, -0...","[-0.3060603737831116, 0.29850369691848755, 0.2..."


In [14]:
# add embedding for movie_id == 0 for unknown movies

_unknow_movie_embedding = [
    np.random.uniform(-1, 1, size=embeddings['synopsis_embeddings'].iloc[0].shape[0]),
    np.random.uniform(-1, 1, size=embeddings['title_embeddings'].iloc[0].shape[0])
]

In [15]:
embeddings.loc[0] = _unknow_movie_embedding
embeddings.sort_index(inplace=True)

In [16]:
synopsis_embeddings = np.stack(embeddings['synopsis_embeddings'])
title_embeddings = np.stack(embeddings['title_embeddings'])

synopsis_embeddings.shape, title_embeddings.shape

((1562, 768), (1562, 768))

In [17]:
# sanity check
assert (synopsis_embeddings[0] == embeddings.loc[0]['synopsis_embeddings']).all()
assert (title_embeddings[0] == embeddings.loc[0]['title_embeddings']).all()

## Build Model
Reference
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/usecases/entertainment-with-pretrained-embeddings.ipynb

### Use Custom Embedding

In [18]:
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.freq_threshold,properties.max_size,properties.num_buckets,properties.start_index,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,1561,movie_id,NaN,NaN
1,user_id,"(Tags.CATEGORICAL, Tags.ID, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,74.0,944.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,943,user_id,NaN,NaN
2,gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,3.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,22.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,16.0,20.0,.//categories/unique.genres.parquet,0.0,0.0,None,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,.//categories/unique.synopsis.parquet,0.0,0.0,None,0.0,0,1561,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,.//categories/unique.title.parquet,0.0,0.0,None,0.0,0,1561,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,178.0,4504.0,.//categories/unique.cast.parquet,0.0,0.0,None,0.0,0,4503,cast,0.0,NaN


In [19]:
# swap synopsis embedding
schema['synopsis'].properties['embedding_sizes'] = {
    'cardinality': float(synopsis_embeddings.shape[0]),
    'dimension': float(synopsis_embeddings.shape[1])
}

# swap title embdeddings
schema['title'].properties['embedding_sizes'] = {
    'cardinality': float(title_embeddings.shape[0]),
    'dimension': float(title_embeddings.shape[1])
}

schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.freq_threshold,properties.max_size,properties.num_buckets,properties.start_index,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,1561,movie_id,NaN,NaN
1,user_id,"(Tags.CATEGORICAL, Tags.ID, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,74.0,944.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,943,user_id,NaN,NaN
2,gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,3.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,22.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,16.0,20.0,.//categories/unique.genres.parquet,0.0,0.0,None,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,768.0,1562.0,.//categories/unique.synopsis.parquet,0.0,0.0,None,0.0,0,1561,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,768.0,1562.0,.//categories/unique.title.parquet,0.0,0.0,None,0.0,0,1561,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,178.0,4504.0,.//categories/unique.cast.parquet,0.0,0.0,None,0.0,0,4503,cast,0.0,NaN


In [42]:
# set schema for train and validation set
train.schema = schema
valid.schema = schema

In [21]:
embed_dims = {}
embed_dims["synopsis"] = synopsis_embeddings.shape[1]
embed_dims["title"] = title_embeddings.shape[1]

embeddings_init={
    "synopsis": mm.TensorInitializer(synopsis_embeddings[:synopsis_embeddings.shape[0]]),
    "title": mm.TensorInitializer(title_embeddings[:title_embeddings.shape[0]]),
}

item_embeddings_block = mm.Embeddings(
    train.schema.select_by_tag(Tags.ITEM),
    infer_embedding_sizes=True,
    embeddings_initializer=embeddings_init,
    trainable={
        'synopsis': False,
        'title': False
        },
    dim=embed_dims,
)

### Model

In [26]:
def build_towers(schema=schema, tower_dim = 64):
    # create user schema using USER tag
    user_schema = schema.select_by_tag(Tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(user_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

    # create item schema using ITEM tag
    item_schema = schema.select_by_tag(Tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema, categorical=item_embeddings_block) # important change
    # create item (candidate) encoder block
    candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

    return mm.TwoTowerModelV2(query, candidate)


In [20]:
# tower_dim = 64

# # create user schema using USER tag
# user_schema = schema.select_by_tag(Tags.USER)
# # create user (query) tower input block
# user_inputs = mm.InputBlockV2(user_schema)
# # create user (query) encoder block
# query = mm.Encoder(user_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

# # create item schema using ITEM tag
# item_schema = schema.select_by_tag(Tags.ITEM)
# # create item (candidate) tower input block
# item_inputs = mm.InputBlockV2(item_schema, categorical=item_embeddings_block)
# # create item (candidate) encoder block
# candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

### model_v1

In [27]:
model_v1 = build_towers()

In [28]:
import tensorflow as tf
model_v1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
history = model_v1.fit(train, validation_data=valid, batch_size=128, epochs=50)

Epoch 1/50
549/549 [==============================] - 50s 79ms/step - loss: 4.7695 - auc_1: 0.5360 - recall_at_10: 0.0922 - ndcg_at_10: 0.0426 - mrr_at_10: 0.0279 - precision_at_10: 0.0092 - regularization_loss: 0.0000e+00 - loss_batch: 4.7669 - val_loss: 4.7594 - val_auc_1: 0.5658 - val_recall_at_10: 0.1234 - val_ndcg_at_10: 0.0567 - val_mrr_at_10: 0.0370 - val_precision_at_10: 0.0123 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.6417
Epoch 2/50
549/549 [==============================] - 19s 32ms/step - loss: 4.6231 - auc_1: 0.5797 - recall_at_10: 0.1417 - ndcg_at_10: 0.0662 - mrr_at_10: 0.0437 - precision_at_10: 0.0142 - regularization_loss: 0.0000e+00 - loss_batch: 4.6207 - val_loss: 4.7017 - val_auc_1: 0.5527 - val_recall_at_10: 0.1355 - val_ndcg_at_10: 0.0628 - val_mrr_at_10: 0.0412 - val_precision_at_10: 0.0136 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.4490
Epoch 3/50
549/549 [==============================] - 19s 32ms/step - loss: 4.5984 - auc_1: 0.58

### model_v2

In [29]:
model_v2 = build_towers()

In [30]:
model_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v2.fit(train, validation_data=valid, batch_size=64, epochs=50)

Epoch 1/50
1097/1097 [==============================] - 67s 55ms/step - loss: 3.9129 - auc_2: 0.6027 - recall_at_10: 0.2871 - ndcg_at_10: 0.1375 - mrr_at_10: 0.0929 - precision_at_10: 0.0287 - regularization_loss: 0.0000e+00 - loss_batch: 3.9127 - val_loss: 3.9576 - val_auc_2: 0.5926 - val_recall_at_10: 0.3273 - val_ndcg_at_10: 0.1566 - val_mrr_at_10: 0.1055 - val_precision_at_10: 0.0327 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.1884
Epoch 2/50
1097/1097 [==============================] - 34s 29ms/step - loss: 3.8223 - auc_2: 0.6163 - recall_at_10: 0.3640 - ndcg_at_10: 0.1789 - mrr_at_10: 0.1235 - precision_at_10: 0.0364 - regularization_loss: 0.0000e+00 - loss_batch: 3.8221 - val_loss: 3.9881 - val_auc_2: 0.5802 - val_recall_at_10: 0.3203 - val_ndcg_at_10: 0.1554 - val_mrr_at_10: 0.1061 - val_precision_at_10: 0.0320 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.2936
Epoch 3/50
1097/1097 [==============================] - 32s 27ms/step - loss: 3.8044 - auc_2

### model_v3

In [31]:
model_v3 = build_towers()

In [32]:
model_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v3.fit(train, validation_data=valid, batch_size=32, epochs=50)

Epoch 1/50
2194/2194 [==============================] - 94s 38ms/step - loss: 3.2582 - auc_3: 0.5588 - recall_at_10: 0.4752 - ndcg_at_10: 0.2300 - mrr_at_10: 0.1567 - precision_at_10: 0.0475 - regularization_loss: 0.0000e+00 - loss_batch: 3.2580 - val_loss: 3.3463 - val_auc_3: 0.5564 - val_recall_at_10: 0.5140 - val_ndcg_at_10: 0.2508 - val_mrr_at_10: 0.1718 - val_precision_at_10: 0.0514 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.5017
Epoch 2/50
2194/2194 [==============================] - 63s 28ms/step - loss: 3.1742 - auc_3: 0.5555 - recall_at_10: 0.5702 - ndcg_at_10: 0.2842 - mrr_at_10: 0.1983 - precision_at_10: 0.0570 - regularization_loss: 0.0000e+00 - loss_batch: 3.1740 - val_loss: 3.2711 - val_auc_3: 0.5687 - val_recall_at_10: 0.5098 - val_ndcg_at_10: 0.2517 - val_mrr_at_10: 0.1741 - val_precision_at_10: 0.0510 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.3516
Epoch 3/50
2194/2194 [==============================] - 63s 28ms/step - loss: 3.1633 - auc_3

KeyboardInterrupt: ignored

### model_v4

In [36]:
model_v4 = build_towers(tower_dim=256)

In [37]:
model_v4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v4.fit(train, validation_data=valid, batch_size=32, epochs=20)

Epoch 1/20
2194/2194 [==============================] - 83s 34ms/step - loss: 3.3626 - auc_5: 0.5355 - recall_at_10: 0.4094 - ndcg_at_10: 0.1906 - mrr_at_10: 0.1256 - precision_at_10: 0.0409 - regularization_loss: 0.0000e+00 - loss_batch: 3.3625 - val_loss: 3.4870 - val_auc_5: 0.5162 - val_recall_at_10: 0.3170 - val_ndcg_at_10: 0.1468 - val_mrr_at_10: 0.0963 - val_precision_at_10: 0.0317 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2081
Epoch 2/20
2194/2194 [==============================] - 59s 26ms/step - loss: 3.4663 - auc_5: 0.5161 - recall_at_10: 0.3688 - ndcg_at_10: 0.2174 - mrr_at_10: 0.1727 - precision_at_10: 0.0369 - regularization_loss: 0.0000e+00 - loss_batch: 3.4661 - val_loss: 4.0444 - val_auc_5: 0.5163 - val_recall_at_10: 0.6392 - val_ndcg_at_10: 0.4860 - val_mrr_at_10: 0.4412 - val_precision_at_10: 0.0639 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.9006
Epoch 3/20
2194/2194 [==============================] - 63s 28ms/step - loss: 3.4638 - auc_5

### model_v5

In [38]:
model_v5 = build_towers(tower_dim=256)

In [39]:
model_v5.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v5.fit(train, validation_data=valid, batch_size=32, epochs=20)

Epoch 1/20
2194/2194 [==============================] - 84s 35ms/step - loss: 4.4061 - auc_6: 0.5083 - recall_at_10: 0.3137 - ndcg_at_10: 0.1432 - mrr_at_10: 0.0927 - precision_at_10: 0.0314 - regularization_loss: 0.0000e+00 - loss_batch: 4.4054 - val_loss: 3.4639 - val_auc_6: 0.5000 - val_recall_at_10: 0.3138 - val_ndcg_at_10: 0.1428 - val_mrr_at_10: 0.0922 - val_precision_at_10: 0.0314 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2828
Epoch 2/20
2194/2194 [==============================] - 59s 26ms/step - loss: 3.4640 - auc_6: 0.5077 - recall_at_10: 0.3134 - ndcg_at_10: 0.1428 - mrr_at_10: 0.0924 - precision_at_10: 0.0313 - regularization_loss: 0.0000e+00 - loss_batch: 3.4639 - val_loss: 3.4641 - val_auc_6: 0.5016 - val_recall_at_10: 0.3138 - val_ndcg_at_10: 0.1428 - val_mrr_at_10: 0.0922 - val_precision_at_10: 0.0314 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2820
Epoch 3/20
2194/2194 [==============================] - 61s 27ms/step - loss: 3.4638 - auc_6

### Embeddings Loader

## Evaluate the Model top-k results

In [ ]:
# Top-K evaluation
from merlin.models.utils.dataset import unique_rows_by_features

candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
candidate_features.head()

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


,movie_id,gender,occupation,genres
0,1,1,1,"[6, 3, 8, 5, 7]"
1,45,1,3,"[2, 5]"
2,357,1,1,"[1, 5]"
3,270,1,2,[1]
4,336,1,15,[4]


In [ ]:
topk = 20
topk_model = model.to_top_k_encoder(candidate_features, k=topk, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


ValueError: ignored

## TMP

In [ ]:
# model = mm.DLRMModel(
#     train.schema,
#     embedding_dim=64,
#     bottom_block=mm.MLPBlock([128, 64]),
#     top_block=mm.MLPBlock([128, 64, 32]),
#     prediction_tasks=mm.BinaryOutput(
#         train.schema.select_by_tag(Tags.TARGET).column_names[0]
#     ),
# )
# model.compile(optimizer="adam")
# model.fit(train, batch_size=1024)

In [41]:
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.freq_threshold,properties.max_size,properties.num_buckets,properties.start_index,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,98.0,1562.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,1561,movie_id,NaN,NaN
1,user_id,"(Tags.CATEGORICAL, Tags.ID, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,74.0,944.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,943,user_id,NaN,NaN
2,gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,3.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,2,gender,NaN,NaN
3,occupation,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,16.0,22.0,/content/drive/MyDrive/ml-twotower-model/data/...,0.0,0.0,None,0.0,0,21,occupation,NaN,NaN
4,genres,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,16.0,20.0,.//categories/unique.genres.parquet,0.0,0.0,None,0.0,0,19,genres,0.0,NaN
5,synopsis,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,768.0,1562.0,.//categories/unique.synopsis.parquet,0.0,0.0,None,0.0,0,1561,synopsis,NaN,NaN
6,title,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,768.0,1562.0,.//categories/unique.title.parquet,0.0,0.0,None,0.0,0,1561,title,NaN,NaN
7,cast,"(Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,178.0,4504.0,.//categories/unique.cast.parquet,0.0,0.0,None,0.0,0,4503,cast,0.0,NaN


In [40]:
# Define Model
QUERY_DROPOUT = 0.1
ITEM_DROPOUT = 0.1

model_v0 = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock(
        [128,64],
        kernel_regularizer=tf.keras.regularizers.l1(3.9e-06),
        bias_regularizer=tf.keras.regularizers.l1(3.9e-06),
        no_activation_last_layer=True,
        dropout=QUERY_DROPOUT
     ),
     item_tower=mm.MLPBlock(
         [128,64],
         kernel_regularizer=tf.keras.regularizers.l1(3.9e-06),
         bias_regularizer=tf.keras.regularizers.l1(3.9e-06),
         no_activation_last_layer=True,
         dropout=ITEM_DROPOUT
     ),
     samplers=[mm.InBatchSampler()]
)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.01
)

In [ ]:
TOP_K = [10,20]

model_v0.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    run_eagerly=False,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[mm.TopKMetricsAggregator.default_metrics(top_ks=TOP_K),
            #  tfr.keras.metrics.MRRMetric(topn=4)
             ],
)
model_v0.fit(
    train,
    validation_data=valid,
    schema=schema,
    batch_size=32,
    epochs=10,
    # shuffle = False
)